# How to make a Supreme Bot

2 main things to understand
* session-user-cart-item relationships
* How to use HTTP post requests and bypass the HTTPOnly system using headers

---
# session-user-cart-item relationships
___

## Session
First thing to do is create a new session

In [2]:
import requests
session = requests.Session()

An important thing to inspect and change is the session headers  
I have not found any reason to believe they are able to check this, but its best you change the `User-Agent` field

In [3]:
session.headers

{'User-Agent': 'python-requests/2.14.2', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [4]:
session.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:64.0) Gecko/20100101 Firefox/64.0'

In [5]:
session.headers

{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.109 Safari/537.36', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

## User
Each user will have their own session  
Each user will have their own cart  
It is import to tie all of these together by consistently using the same session for all GET and POST requests  

## Adding items to cart

You will need to use the the `add` endpoint example like this https://www.supremenewyork.com/shop/{item_code}/add  
With form data and proper headers  
Its all about replicating what your browser is doing as much as possible
  

How to find the {item_code}?  
Have a look around on an item page by finding where the {item_code} from the url exists on the webpage 

## How to use the Network tab to solve this problem
The process will be this throughout the project.  

1. You need to inspect the headers and form data to see what is being passed.  
<img src="form_data.png">
2. You need to find where that data exists on the web page  
(See how I took the st value from the form data and search where it was in the webpage)
<img src="found_element.png">
3. Create functions to scrape that data from the webpage
4. Replicate those request header and form data packets with your POST request
<img src="header_example.png">
<img src="form_data_close.png">
5. Read response headers to see how you performed and continue logically off that  
(Specifically the 'set-cookie' from add to cart and 'queued' from checkout)
<img src="set_cookie_close.png">

---
# How to beat HTTPOnly
---
From a high level there are several things you need to nail down that interweave with  
  
* Request Headers
    * X-CSRF code
    * cookie
        * cart
        * pure_cart
        * \_supreme_sess
* Response Headers
    * Taking the cookie returned and placing it into the next post request
* Checkout form data
    * Solving a captcha and placing that into the form data for each checkout

## X-CSRF Code
I think this is why you need to use the same session across all of the GET and POST requests  
so you get the proper X-CSRF code into the header.  
  
Just scrape this from the item page you want to add to cart and your checkout page 
  
<img src="csrf.png">

## Cookie parts
There a a couple of parts of the cookie that need to be separately created then assembled
* \_supreme_sess
* cart
* pure_cart
* request_method

cart and pure_cart are encoded. Use https://www.urldecoder.org/ to start figuring out what's going on and reverse engineer it.  

request_method seems to be the HTTP request you made. I just hardcode it Post because why would your bot be deleting anything.  

pure_cart is finicky I would suggest including it but it doesnt always return in the response headers and isn't needed in the request headers. I would 100% put it in the POST checkout.  

Throw all of these in one cookie named`['set-cookie']` separated by commas


## \_supreme_sess pass off
Through my testing on incognito mode I discovered you don't need to send any pookies ever and don't need to send any cookies on the first add to cart

Here is a write up of the process.  

1 Add to Cart No cookie  
1 add to cart 200 and cookie = response.headers['cookie']  
2 Add to Cart 1 cookie  
2 200 and cookie = response.headers['cookie']  
3 Add to Cart 2 cookie  
3 200 and cookie = response.headers['cookie']  
.  
.  
.  
Nth Add to Cart N-1 cookie  
Nth 200 and cookie = response.headers['cookie']  
  
I also show how this process works through images

<img src="request_incognito.png">
  
But you do need to take the response header (didn't get the whole header)
  
<img src="response_incognito.png">
  
And Take the set-cookie and put back in the next request header
  
<img src="remove_cart_request_incognito.png">
  
That's what the website does so make your application do the same thing too. It took me $450 to figure this out so be thankful. This is only 1 element of the cookie

## Solving Captchas
I use http://www.deathbycaptcha.com/user/login, but you can really use any captcha solver. I almost forgot to write about this because of how simple this is.  

For every checkout you need a g-captcha-repsonse code that is less then 120 seconds old. Just put it into your checkout form data. There is plenty of info to learn from googling how the captcha system works

## Threading
Threading is a huge issue I am unqualified to talk about but just think all this shit has to work at the same time like an orchestra. Good thing Rudy and I were in band together.

## Tips

Overall process
Get item page-->Scrape item page for values--> Post Add item to cart--> Scrape Post response--> Post Checkout

Lots of webscraping

Time is always important. Either time of the day or how long something has been going or existed

Program this like the supreme site is doing to crash